Joule Conversational Search
to learn 
https://python.langchain.com/v0.2/docs/tutorials/
-Basics 
<!-- Build a Simple LLM Application with LCEL -->
<!-- Build a Chatbot -->
<!-- Build vector stores and retrievers -->
Build an Agent

-Working with external knowledge
Build a Retrieval Augmented Generation (RAG) Application
Build a Conversational RAG Application
Build a PDF ingestion and Question/Answering system


Automating manual testing process
-how to use free model on langchain
-ragas: how to use the free model instead of openai by default
-relating metrics to those in excel file

# Build a Simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

In [5]:
import getpass 
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4')

 ········


In [7]:
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x16858e040>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1685929d0>, root_client=<openai.OpenAI object at 0x1059ad6a0>, root_async_client=<openai.AsyncOpenAI object at 0x16858e070>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content='Translate the following from English to French'),
    HumanMessage(content='hi!')
]

model.invoke(messages)

AIMessage(content='salut!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49928e79-6aea-45f5-9716-0dcaff45da2d-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser() # use this to parse only the response content without the other details
result = model.invoke(messages)
parser.invoke(result)

'salut!'

instead of doing it sequentially, can use a chain instead 

In [20]:
chain = model | parser
chain.invoke(messages)

'salut!'

# Prompt Template

usually prompts are user inputs, but to make it a comprehensive prompt where a user input is just 1 part of the prompt, should use a prompt template

In [30]:
from langchain_core.prompts import ChatPromptTemplate

# has 2 parts - system and user
# - this will become SystemMessage and HumanMessage

system_template = "Translate the following into {language}"
text_template = "{text}"
# the str in {} will become the dict keys for user inputs during invoke

# create prompt template, combination of system template and simpler template for user input template

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system',system_template),
        ('user',text_template)
    ]
)

result = prompt_template.invoke(
    {
        'language':'malay',
        'text':"where are we eating dinner tonight?"
    }
)
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into malay'), HumanMessage(content='where are we eating dinner tonight?')])

In [32]:
# to access the messages directly
result.to_messages()

[SystemMessage(content='Translate the following into malay'),
 HumanMessage(content='where are we eating dinner tonight?')]

chain together components with LCEL (Langchain Expression Language)

In [34]:
chain = prompt_template | model | parser
result = chain.invoke(
    {
        'language': 'Malay',
        'text': 'Lets go to the zoo where there are many elephants playing with tigers'
    }
)

result

'Mari pergi ke zoo di mana terdapat banyak gajah bermain dengan harimau.'

using chain.invoke will directly give the output message, instead of model.invoke which will return an AIMessage which you then need to pass thru StrOutputParser

# Summary of Chat usage

1. Create PromptTemplate
2. Initialise model
3. Create StrOutputParser
4. Create chain
5. chain.invoke()

Notes on SystemMessage vs HumanMessage vs AIMessage

SystemMessage

Message for priming AI behavior.
The system message is usually passed in as the first of a sequence of input messages.

HumanMessages

Message from a human.
HumanMessages are messages that are passed in from a human to the model.

AIMessage

Message from an AI.
AIMessage is returned from a chat model as a response to a prompt.
This message represents the output of the model and consists of both the raw output as returned by the model together standardized fields (e.g., tool calls, usage metadata) added by the LangChain framework.

In [ ]:
# e.g. SystemMessage should be used to "instruct/inform" the chat model about its task, characteristics etc

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant! Your name is Bob."
    ),
    HumanMessage(
        content="What is your name?"
    )
]

# Define a chat model and invoke it with the messages
print(model.invoke(messages))